In [1]:
import spotipy

from spotipy.oauth2 import SpotifyClientCredentials
import csv
import re
import pandas as pd

In [2]:
client_id = "43c1aefc29484fab8b76593469ab4c44"
client_secret = "df001dead9514c7e8f65ab4dbfaee2ba"

In [3]:
# Autenticación
manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=manager)

### Global - EEUU - Chile

In [49]:
def limpiar_titulo_youtube(titulo):
    """
    Limpia el título eliminando etiquetas como [Official Video], (Audio), etc.
    """
    titulo_limpio = re.sub(r"\[.*?\]|\(.*?\)|\{.*?\}", "", titulo)  # Eliminar etiquetas
    titulo_limpio = re.sub(r"\b(ft\.|feat\.|featuring)\b", "", titulo_limpio, flags=re.IGNORECASE)  # ft./feat.
    return titulo_limpio.strip()


In [50]:
yt_chile = pd.read_csv("videos_chile.csv")
yt_global = pd.read_csv("videos_global.csv")
yt_eeuu = pd.read_csv("videos_eeuu.csv")

yt_merged = pd.concat([yt_global, yt_eeuu, yt_chile]).drop_duplicates().reset_index(drop=True)
yt_merged

,Titulo,Vistas,Likes,Comentarios,Fecha,Tags
0,"KAROL G, Feid, DFZM ft. Ovy On The Drums, J Ba...",15598352,883849,73874,2024-11-08T00:00:06Z,['#57']
1,YoungBoy Never Broke Again - Sneaking [Officia...,1029066,75819,3901,2024-11-08T03:41:23Z,[]
2,YoungBoy Never Broke Again - Killa Season [Off...,1164882,81449,4392,2024-11-08T04:16:17Z,[]
3,YoungBoy Never Broke Again - Missing Everythin...,1132351,86119,6341,2024-11-08T02:20:12Z,[]
4,YoungBoy Never Broke Again - Catch Me [Officia...,812399,55568,2963,2024-11-08T02:20:09Z,[]
...,...,...,...,...,...,...
82,"LIT killah, TULI - MALA SUERTE (Official Video)",514359,51016,1321,2024-11-08T22:00:07Z,"['#MALASUERTE', '#TULI']"
83,Pac-Man vs Kirby. Épicas Batallas de Rap del F...,428853,50316,4518,2024-11-07T20:00:06Z,[]
84,"Valentino merlo, Salastkbron - Me voy (Officia...",713734,27106,1267,2024-11-07T23:00:08Z,[]
85,Lo Grande Que Es Perdonar (feat. Gilberto Sant...,680358,3230,30,2024-11-01T04:00:21Z,[]


In [51]:
# Aplicar la limpieza al DataFrame
yt_merged['Titulo'] = yt_merged['Titulo'].apply(limpiar_titulo_youtube)

In [52]:
yt_merged

,Titulo,Vistas,Likes,Comentarios,Fecha,Tags
0,"KAROL G, Feid, DFZM ft. Ovy On The Drums, J Ba...",15598352,883849,73874,2024-11-08T00:00:06Z,['#57']
1,YoungBoy Never Broke Again - Sneaking,1029066,75819,3901,2024-11-08T03:41:23Z,[]
2,YoungBoy Never Broke Again - Killa Season,1164882,81449,4392,2024-11-08T04:16:17Z,[]
3,YoungBoy Never Broke Again - Missing Everything,1132351,86119,6341,2024-11-08T02:20:12Z,[]
4,YoungBoy Never Broke Again - Catch Me,812399,55568,2963,2024-11-08T02:20:09Z,[]
...,...,...,...,...,...,...
82,"LIT killah, TULI - MALA SUERTE",514359,51016,1321,2024-11-08T22:00:07Z,"['#MALASUERTE', '#TULI']"
83,Pac-Man vs Kirby. Épicas Batallas de Rap del F...,428853,50316,4518,2024-11-07T20:00:06Z,[]
84,"Valentino merlo, Salastkbron - Me voy",713734,27106,1267,2024-11-07T23:00:08Z,[]
85,Lo Grande Que Es Perdonar,680358,3230,30,2024-11-01T04:00:21Z,[]


In [53]:
def buscar_en_spotify(titulo):

    data = sp.search(q = titulo, type = "track", limit = 1)

    if data['tracks']['items']:
        cancion = data['tracks']['items'][0]
        artista_id = cancion['artists'][0]['id']  # ID del artista
        artista_data = sp.artist(artista_id)
        return {
            'Titulo': cancion['name'],
            'Artista': cancion['artists'][0]['name'],
            'Popularidad': cancion['popularity'],
            'Generos': artista_data['genres']
            }
    return {'Titulo': None, 'Artista': None, 'Popularidad': None, 'Generos': None}

# Buscar cada canción en Spotify y agregar los resultados al DataFrame
resultados = yt_merged['Titulo'].apply(buscar_en_spotify)
df_spotify = pd.DataFrame(resultados.tolist())
df = pd.concat([yt_merged, df_spotify], axis=1)

In [54]:
df_spotify

,Titulo,Artista,Popularidad,Generos
0,+57,KAROL G,84,"[reggaeton, reggaeton colombiano, urbano latino]"
1,Sneaking,YoungBoy Never Broke Again,59,"[baton rouge rap, rap]"
2,Killa Season,YoungBoy Never Broke Again,59,"[baton rouge rap, rap]"
3,Missing Everything,YoungBoy Never Broke Again,57,"[baton rouge rap, rap]"
4,Catch Me,YoungBoy Never Broke Again,54,"[baton rouge rap, rap]"
...,...,...,...,...
82,MALA SUERTE,LIT killah,66,"[argentine hip hop, trap argentino, trap latin..."
83,Pac-Man vs Kirby,Keyblade,41,"[anime latino, latin viral rap, rap anime]"
84,Me Voy,Valentino Merlo,62,[]
85,Lo Grande Que Es Perdonar,Agrupación Marilyn,33,"[cumbia chilena, cumbia paraguaya, cumbia vill..."


In [ ]:
df_spotify["Titulo"] = yt_merged['Titulo']

In [56]:
final_merge = pd.merge(yt_merged, df_spotify, on= "Titulo", how= "inner")

In [58]:
final_merge = final_merge.drop_duplicates(subset=['Titulo']).reset_index(drop = True)

In [59]:
final_merge.to_csv('youtube_en_spotify.csv', index=False)


### Global

In [4]:
def limpiar_titulo_youtube(titulo):
    """
    Limpia el título eliminando etiquetas como [Official Video], (Audio), etc.
    """
    titulo_limpio = re.sub(r"\[.*?\]|\(.*?\)|\{.*?\}", "", titulo)  # Eliminar etiquetas
    titulo_limpio = re.sub(r"\b(ft\.|feat\.|featuring)\b", "", titulo_limpio, flags=re.IGNORECASE)  # ft./feat.
    return titulo_limpio.strip()


In [5]:
yt_global = pd.read_csv("videos_global.csv")
yt_global['Titulo'] = yt_global['Titulo'].apply(limpiar_titulo_youtube)

def buscar_en_spotify(titulo):

    data = sp.search(q = titulo, type = "track", limit = 1)

    if data['tracks']['items']:
        cancion = data['tracks']['items'][0]
        artista_id = cancion['artists'][0]['id']  # ID del artista
        artista_data = sp.artist(artista_id)
        return {
            'Titulo': cancion['name'],
            'Artista': cancion['artists'][0]['name'],
            'Popularidad': cancion['popularity'],
            'Generos': artista_data['genres']
            }
    return {'Titulo': None, 'Artista': None, 'Popularidad': None, 'Generos': None}

# Buscar cada canción en Spotify y agregar los resultados al DataFrame
resultados = yt_global['Titulo'].apply(buscar_en_spotify)
df_spotify = pd.DataFrame(resultados.tolist())
df = pd.concat([yt_global, df_spotify], axis=1)

In [6]:
df_spotify["Titulo"] = yt_global['Titulo']
final_merge = pd.merge(yt_global, df_spotify, on= "Titulo", how= "inner")
final_merge = final_merge.drop_duplicates(subset=['Titulo']).reset_index(drop = True)
final_merge.to_csv('youtube_en_spotify_global.csv', index=False)